In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

import spicy

from sklearn.inspection import permutation_importance
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.model_selection import learning_curve

import pickle

In [2]:
df = pd.read_csv(r'C:\Users\Admin\Documents\marianneSimplon\simplon\prediction_CO2\data\model.csv',delimiter=',', decimal='.')

In [3]:
df = df.drop(['Unnamed: 0','ListOfAllPropertyUseTypes'], axis=1)

In [4]:
df

,BuildingType,PrimaryPropertyType,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofFloors,PropertyGFAParking,LargestPropertyUseType,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,SourceEUI(kBtu/sf),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),DefaultData,ComplianceStatus,TotalGHGEmissions,GHGEmissionsIntensity
0,NonResidential,Hotel,7,DOWNTOWN,47.61220,-122.33799,1927,12,0,Hotel,Rien,28444.075817,182.500000,7.456910e+06,2003882.00,3.946027e+06,False,Compliant,249.98,2.83
1,NonResidential,Hotel,7,DOWNTOWN,47.61317,-122.33393,1996,11,15064,Hotel,Parking,15064.000000,176.100006,8.664479e+06,0.00,3.242851e+06,False,Compliant,295.86,2.86
2,NonResidential,Hotel,7,DOWNTOWN,47.61393,-122.33810,1969,41,196718,Hotel,Rien,28444.075817,241.899994,7.393711e+07,21566554.00,4.952666e+07,False,Compliant,2089.28,2.19
3,NonResidential,Hotel,7,DOWNTOWN,47.61412,-122.33664,1926,10,0,Hotel,Rien,28444.075817,216.199997,6.946800e+06,2214446.25,2.768924e+06,False,Compliant,286.43,4.67
4,NonResidential,Hotel,7,DOWNTOWN,47.61375,-122.34047,1980,18,62000,Hotel,Parking,68009.000000,211.399994,1.465650e+07,0.00,5.368607e+06,False,Compliant,505.01,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,Nonresidential COS,Office,2,GREATER DUWAMISH,47.56722,-122.31154,1990,1,0,Office,Rien,28444.075817,161.699997,9.430032e+05,0.00,5.242709e+05,True,Error - Correct Default Data,20.94,1.70
3371,Nonresidential COS,Other,2,DOWNTOWN,47.59625,-122.32283,2004,1,0,Other - Recreation,Rien,28444.075817,114.199997,1.053706e+06,0.00,3.965461e+05,False,Compliant,32.17,2.01
3372,Nonresidential COS,Other,7,MAGNOLIA / QUEEN ANNE,47.63644,-122.35784,1974,1,0,Other - Recreation,Fitness Center/Health Club/Gym,5574.000000,744.799988,6.053764e+06,0.00,1.792159e+06,False,Compliant,223.54,16.99
3373,Nonresidential COS,Mixed Use Property,1,GREATER DUWAMISH,47.52832,-122.32431,1989,1,0,Other - Recreation,Fitness Center/Health Club/Gym,6501.000000,105.300003,7.828413e+05,0.00,3.488702e+05,False,Compliant,22.11,1.57


# Processing

In [6]:
y = df['TotalGHGEmissions']
X = df.drop('TotalGHGEmissions', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [7]:
y.describe()

count     3375.000000
mean       119.759682
std        538.189041
min          0.000000
25%          9.565000
50%         34.040000
75%         94.610000
max      16870.980000
Name: TotalGHGEmissions, dtype: float64

In [8]:
X.shape

(3375, 19)

# Pipelines

In [9]:
numeric_transformer = Pipeline(steps=[
       ('scaler', MinMaxScaler())
])
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first',handle_unknown = 'ignore'))
])

In [10]:
cat = X.select_dtypes(include=["object"])
non_cat = X.select_dtypes(exclude=["object"])
noncatY = df.select_dtypes(exclude=["object"])

cat['CouncilDistrictCode'] = non_cat['CouncilDistrictCode']
boolean = non_cat['DefaultData']

<ipython-input-10-88fa7983512d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['CouncilDistrictCode'] = non_cat['CouncilDistrictCode']


In [59]:
numeric_features = non_cat.columns.tolist()
categorical_features = cat.columns.tolist()
boolean_features = ['DefaultData']

In [60]:
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
#    ,('categorical', categorical_transformer, categorical_features)
]) 

In [61]:
preprocess = Pipeline(steps=[('preprocessor', preprocessor)])

In [47]:
# pd.DataFrame.sparse.from_spmatrix(training_transformed)

,0,1,2,3,4,5,6,7,8,9,...,161,162,163,164,165,166,167,168,169,170
0,0.333333,0.531492,0.471931,0.591304,0.126984,0.0,0.044449,0.032137,0.004109,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.000000,0.424097,0.411571,0.000000,0.047619,0.0,0.009407,0.063053,0.005780,0.004085,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.500000,0.684289,0.503054,0.260870,0.047619,0.0,0.044449,0.041718,0.001611,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.666667,0.862852,0.457051,0.939130,0.063492,0.0,0.044449,0.083435,0.030449,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.000000,0.439702,0.417911,0.095652,0.095238,0.0,0.003963,0.042214,0.004792,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526,0.333333,0.537065,0.460142,0.260870,0.047619,0.0,0.044449,0.030191,0.003116,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2527,1.000000,0.468684,0.364048,0.034783,0.047619,0.0,0.008849,0.140000,0.009949,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2528,0.666667,0.896935,0.488174,0.452174,0.031746,0.0,0.024643,0.050191,0.004618,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2529,0.166667,0.280732,0.409947,0.400000,0.031746,0.0,0.044449,0.049695,0.004000,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [62]:
X_train_num = X_train[numeric_features]
X_train_num

,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofFloors,PropertyGFAParking,SecondLargestPropertyUseTypeGFA,SourceEUI(kBtu/sf),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),DefaultData,GHGEmissionsIntensity
2598,3,47.62459,-122.32175,1968,8,0,28444.075817,84.199997,1.937882e+06,0.0000,1092145.0,False,0.89
2439,7,47.59954,-122.33327,1900,3,0,6020.000000,165.199997,2.726127e+06,500752.8125,572200.0,False,4.33
923,4,47.66023,-122.31581,1930,3,0,28444.075817,109.300003,7.597565e+05,0.0000,716279.0,False,0.24
3082,5,47.70188,-122.32459,2008,4,0,28444.075817,218.600006,1.436022e+07,0.0000,3963184.0,False,11.38
778,7,47.60318,-122.33206,1911,6,0,2536.000000,110.599999,2.259986e+06,0.0000,644057.0,False,3.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,3,47.62589,-122.32400,1930,3,0,28444.075817,79.099998,1.469579e+06,0.0000,278629.0,False,1.90
3264,7,47.60994,-122.34234,1904,3,0,5663.000000,366.799988,4.691944e+06,0.0000,3599281.0,False,1.60
1653,5,47.70983,-122.31865,1952,2,0,15770.000000,131.500000,2.177737e+06,0.0000,1340993.0,False,1.38
2607,2,47.56610,-122.33358,1946,2,0,28444.075817,130.199997,1.886367e+06,0.0000,1126394.0,False,1.26


In [71]:
training_transformed = preprocess.fit_transform(X_train_num)
pd.DataFrame(training_transformed, columns=X_train_num.columns)

,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofFloors,PropertyGFAParking,SecondLargestPropertyUseTypeGFA,SourceEUI(kBtu/sf),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),DefaultData,GHGEmissionsIntensity
0,0.333333,0.531492,0.471931,0.591304,0.126984,0.0,0.044449,0.032137,0.004109,0.000000,0.001662,0.0,0.026107
1,1.000000,0.424097,0.411571,0.000000,0.047619,0.0,0.009407,0.063053,0.005780,0.004085,0.000871,0.0,0.127017
2,0.500000,0.684289,0.503054,0.260870,0.047619,0.0,0.044449,0.041718,0.001611,0.000000,0.001090,0.0,0.007040
3,0.666667,0.862852,0.457051,0.939130,0.063492,0.0,0.044449,0.083435,0.030449,0.000000,0.006032,0.0,0.333822
4,1.000000,0.439702,0.417911,0.095652,0.095238,0.0,0.003963,0.042214,0.004792,0.000000,0.000980,0.0,0.099443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526,0.333333,0.537065,0.460142,0.260870,0.047619,0.0,0.044449,0.030191,0.003116,0.000000,0.000424,0.0,0.055735
2527,1.000000,0.468684,0.364048,0.034783,0.047619,0.0,0.008849,0.140000,0.009949,0.000000,0.005478,0.0,0.046935
2528,0.666667,0.896935,0.488174,0.452174,0.031746,0.0,0.024643,0.050191,0.004618,0.000000,0.002041,0.0,0.040481
2529,0.166667,0.280732,0.409947,0.400000,0.031746,0.0,0.044449,0.049695,0.004000,0.000000,0.001714,0.0,0.036961


# Model Selection

### Dummy

In [74]:
pipe_dummy = Pipeline(steps = [
                ('preprocessor', preprocessor),
                ('dummy', DummyRegressor())
           ])

In [75]:
pipe_dummy.fit(X_train,y_train)
pipe_dummy.score(X_train,y_train)
# cv_scores = cross_val_score(pipe_dummy,X_train, y_train,cv=5,scoring='recall')
# cv_scores.mean()

0.0

In [77]:
pipe_dummy_ypred = pipe_dummy.predict(X_test)
print(f"MSE : {mean_squared_error(y_test, pipe_dummy_ypred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, pipe_dummy_ypred))}")
print(f"MAE : {mean_absolute_error(y_test, pipe_dummy_ypred)}")

MSE : 330535.0519508091
RMSE : 574.921778984593
MAE : 139.04255026376367


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


### Regression lineaire

In [15]:
pipe_reglin = Pipeline(steps = [
                ('preprocessor', preprocessor),
                ('linear', LinearRegression())
           ])

In [16]:
pipe_reglin.fit(X_train,y_train)
pipe_reglin.score(X_train,y_train)

0.9168591393020452

In [17]:
pipe_reglin_ypred = pipe_reglin.predict(X_test)
print(f"MSE : {mean_squared_error(y_test, pipe_reglin_ypred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, pipe_reglin_ypred))}")
print(f"MAE : {mean_absolute_error(y_test, pipe_reglin_ypred)}")

MSE : 17686.78409955955
RMSE : 132.99166928631112
MAE : 64.88483329225078


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# Permutation

In [18]:
# Perform Permutation
permutation_score = permutation_importance(pipe_reglin, 
                                           X_test, y_test,
                                           scoring='neg_mean_absolute_error',
                                           random_state=3,
                                           n_repeats=100) 

# Unstack results
importance_df = pd.DataFrame(np.vstack((X.columns,
                                        permutation_score.importances_mean)).T) 

importance_df.columns=['feature','feature importance']

# Order by importance
importance_df.sort_values(by="feature importance", ascending = False) 

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

...

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserW

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,feature,feature importance
13,SiteEnergyUseWN(kBtu),98.8645
18,GHGEmissionsIntensity,49.8671
9,LargestPropertyUseType,48.9185
1,PrimaryPropertyType,41.7929
12,SourceEUI(kBtu/sf),35.561
15,Electricity(kBtu),22.0977
7,NumberofFloors,21.6372
14,SteamUse(kBtu),16.2648
2,CouncilDistrictCode,7.81821
3,Neighborhood,6.9013


'DefaultData','Longitude','ComplianceStatus','PropertyGFAParking','SecondLargestPropertyUseTypeGFA','YearBuilt','Latitude','SecondLargestPropertyUseType'

# VIF

In [93]:
numeric_transformerVIF = Pipeline(steps=[
       ('scaler', MinMaxScaler())
])

In [94]:
# cat = X.select_dtypes(include=["object"])
# non_cat = X.select_dtypes(exclude=["object"])
noncatY = df.select_dtypes(exclude=["object"])

# cat['CouncilDistrictCode'] = non_cat['CouncilDistrictCode']
# boolean = non_cat['DefaultData']

In [95]:
numeric_featuresVIF = noncatY.columns.tolist()

In [96]:
preprocessorVIF = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformerVIF, numeric_featuresVIF)
]) 

In [97]:
preprocessVIF = Pipeline(steps=[('preprocessorVIF', preprocessorVIF)])

In [98]:
# X_train_num = X_train[numeric_featuresVIF]
# X_train_num

In [99]:
# pd.DataFrame.sparse.from_spmatrix(training_transformed)

In [102]:
training_transformedVIF = preprocessVIF.fit_transform(noncatY)
pd.DataFrame(training_transformedVIF, columns=noncatY.columns)

,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofFloors,PropertyGFAParking,SecondLargestPropertyUseTypeGFA,SourceEUI(kBtu/sf),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),DefaultData,TotalGHGEmissions,GHGEmissionsIntensity
0,1.000000,0.481594,0.394549,0.234783,0.121212,0.000000,0.041418,0.069656,0.015811,0.014850,0.006005,0.0,0.014817,0.083016
1,1.000000,0.485726,0.415554,0.834783,0.111111,0.029387,0.021935,0.067214,0.018372,0.000000,0.004935,0.0,0.017537,0.083896
2,1.000000,0.488965,0.393980,0.600000,0.414141,0.383759,0.041418,0.092328,0.156775,0.159819,0.075375,0.0,0.123839,0.064242
3,1.000000,0.489774,0.401533,0.226087,0.101010,0.000000,0.041418,0.082519,0.014730,0.016410,0.004214,0.0,0.016978,0.136990
4,1.000000,0.488198,0.381718,0.695652,0.181818,0.120950,0.099030,0.080687,0.031077,0.000000,0.008170,0.0,0.029934,0.084482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,0.166667,0.289945,0.531394,0.782609,0.010101,0.000000,0.041418,0.061718,0.002000,0.000000,0.000798,1.0,0.001241,0.049868
3371,0.166667,0.413634,0.472983,0.904348,0.010101,0.000000,0.041418,0.043588,0.002234,0.000000,0.000604,0.0,0.001907,0.058962
3372,1.000000,0.584874,0.291850,0.643478,0.010101,0.000000,0.008116,0.284275,0.012836,0.000000,0.002727,0.0,0.013250,0.498387
3373,0.000000,0.124201,0.465325,0.773913,0.010101,0.000000,0.009466,0.040191,0.001660,0.000000,0.000531,0.0,0.001311,0.046055


In [103]:
Xprepro = pd.DataFrame(training_transformedVIF, columns=noncatY.columns)

In [49]:
# Xprepro = pd.DataFrame(preprocess.fit_transform(X))

In [104]:
Xprepro = pd.DataFrame(Xprepro, columns=noncatY.columns)

In [105]:
Xprepro.shape

(3375, 14)

In [106]:
vif_df = pd.DataFrame()
vif_df["vif_index"] = [vif(Xprepro.values, i) for i in range(Xprepro.shape[1])]
vif_df["features"] = Xprepro.columns
vif_df

,vif_index,features
0,5.612186,CouncilDistrictCode
1,10.708534,Latitude
2,6.102476,Longitude
3,5.028966,YearBuilt
4,2.707548,NumberofFloors
5,1.523874,PropertyGFAParking
6,2.851095,SecondLargestPropertyUseTypeGFA
7,4.408256,SourceEUI(kBtu/sf)
8,8.104120,SiteEnergyUseWN(kBtu)
9,2.221709,SteamUse(kBtu)


In [ ]:
'CouncilDistrictCode','Latitude','Longitude','SiteEnergyUseWN(kBtu)'

# Performance evaluation

In [88]:
train_sizes, train_scores, test_scores = learning_curve(estimator = pipe_reglin,
                                          X = X,
                                          y = y,
                                          train_sizes = [5,10,20,30,40,50,100,200,300,400],
                                          cv = 5,
                                          scoring='neg_mean_absolute_error',
                                          shuffle = True,
                                          random_state=0)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-pac

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1, 2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\preprocessing\_e